In [1]:
import re
import numpy as np
import os

import torch
from transformers import pipeline
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
from compel import Compel, ReturnedEmbeddingsType

import weaviate


client = weaviate.Client("http://localhost:8080")

# load huggingface key string from huggingface_token.txt
with open("huggingface_token.txt", "r") as f:
    huggingface_token = f.read().strip()


llm = pipeline(
    "text-generation",
    model="meta-llama/Llama-2-13b-hf",
    device_map="auto",
    torch_dtype=torch.float16,
    token=huggingface_token
)

diffuser_base = StableDiffusionXLPipeline.from_single_file("models/cmpcrtx-000008.safetensors", torch_dtype=torch.float16).to("cuda")
#diffuser_base.load_lora_weights("./models/cmpvsn-000005.safetensors")

diffuser_refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-xl-refiner-1.0", text_encoder_2=diffuser_base.text_encoder_2, vae=diffuser_base.vae, torch_dtype=torch.float16, variant="fp16", use_safetensors=True, device_map="auto")

compel = Compel(tokenizer=[diffuser_base.tokenizer, diffuser_base.tokenizer], text_encoder=[diffuser_base.text_encoder, diffuser_base.text_encoder_2], returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED, requires_pooled=[False, True])



question_classifier = pipeline(
    "text-classification", model="shahrukhx01/question-vs-statement-classifier"
)

className = "Thought"

c:\Users\Lars\Documents\Folkwang\Semester6\Bachelor\app\ai\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:29<00:00,  9.67s/it]
c:\Users\Lars\Documents\Folkwang\Semester6\Bachelor\app\ai\.venv\lib\site-packages\transformers\utils\hub.py:373: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 5/5 [00:07<00:00,  1.47s/it]


In [2]:
def generateText(prompt, do_sample=True, max_new_tokens=64, temperature=0.9, top_p=0.9, top_k=50, cleanup=True):
    response = llm(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        return_full_text=False,
        repetition_penalty=1.2,
    )[0]["generated_text"]

    # split the text at the last period, question mark, or exclamation point to get rid of incomplete sentences but keep the punctuation
    if cleanup:
        response = re.split(r"([^.!?]+$)", response)[0].strip()

    # if the response is empty, try again
    if response == "":
        return generateText(
            prompt,
            do_sample=do_sample,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
        )
    
    torch.cuda.empty_cache()

    return response


In [3]:
from datetime import datetime
import json

def get_thought_score(date, outs, length, newestDate, oldestDate):
    # Set the weights for each factor (you can adjust these as desired)
    length_weight = 1
    references_weight = 3
    recency_weight = 7

    # Calculate the score for each factor
    length_score = length / 256  # Normalize the length between 0 and 1
    references_score = outs / 5  # Normalize the references between 0 and 1

    recency_score = 1
    # interpolate the date between the newest and oldest date (0 and 1)
    if newestDate != oldestDate:
        recency_score = (date - oldestDate) / (newestDate - oldestDate)
    else:
        recency_score = 1


    # Calculate the composite score by combining the weighted factors
    composite_score = (length_score * length_weight) + (references_score * references_weight) + (recency_score * recency_weight)

    return composite_score

def get_context_recursive(thought, depth = 2, ctxt=[]):
    if depth == 0:
        return ctxt
    
    if thought["in"]:
        for parent in thought["in"]:
            if "_additional" in parent:
                result = client.data_object.get_by_id(parent["_additional"]["id"], class_name="Thought")
            else:
                result = client.data_object.get_by_id(parent["beacon"].split("localhost/")[1], class_name="Thought")  

            if result:
                ctxt = get_context_recursive(result["properties"], depth - 1, ctxt)
                ctxt.append({"generated": result["properties"]["generated"], "text": result["properties"]["text"]})

    return ctxt

def get_random_thought(excluded_ids = ["0"]):
    operands = []
    for id in excluded_ids:
        operands.append(
            {
                "path": ["id"],
                "operator": "NotEqual",
                "valueText": id,
            }
        )

    # print(operands)


    result = (
        client.query.get(
            className,
            [
                "text",
                "category",
                "generated",
                "in {... on " + className + "{_additional {id}}}",
                "out {... on " + className + "{_additional {id}}}",
            ],
        )
        .with_additional(["id", "creationTimeUnix"])
        .with_where({
            "operator": "And",
            "operands": operands
        })
        .with_sort({"path": ["_creationTimeUnix"], "order": "desc"})
        .with_limit(10)
        .do()
    )

    # print(json.dumps(result, indent=2))

    if not result["data"]["Get"][className]:
        return None

    # calculate hotness for each thought
    newest_date = datetime.fromtimestamp(float(result["data"]["Get"][className][0]["_additional"]["creationTimeUnix"]) / 1000)
    oldest_date = datetime.fromtimestamp(float(result["data"]["Get"][className][-1]["_additional"]["creationTimeUnix"]) / 1000)

    for thought in result["data"]["Get"][className]:
        date = datetime.fromtimestamp(float(thought["_additional"]["creationTimeUnix"]) / 1000)
        numOuts = 0
        if thought['out']:
            numOuts = len(thought['out'])
        if thought['text']:
            thought["score"] = get_thought_score(date, numOuts, len(thought['text']), newest_date, oldest_date)
        else:
            thought["score"] = get_thought_score(date, numOuts, 0, newest_date, oldest_date)
    #print(json.dumps(result, indent=2))


    # pick a random thought based on their score as a weight
    thoughts = []
    scores = []
    for thought in result["data"]["Get"][className]:
        thoughts.append(thought)
        scores.append(thought["score"])
    scores = np.array(scores)
    scores /= scores.sum()
    result = np.random.choice(thoughts, p=scores)

    context = get_context_recursive(result, depth = 2, ctxt=[])

    cleanedResult = {
        "text": result["text"],
        "in": result["in"][0]["_additional"]["id"]
        if result["in"]
        else None,
        "category": result["category"],
        "generated": result["generated"],
        "id": result["_additional"]["id"],
        "creationTimeUnix": result["_additional"]["creationTimeUnix"],
        "score": result["score"],
        "context": context
    }
    # print(json.dumps(cleanedResult, indent=2))

    return cleanedResult

In [4]:
def task_classify_question(thought):
    print("Task: classifying question...")
    classification = question_classifier(thought["text"])
    # print("Classification:", classification)
    return classification[0]["label"] == "LABEL_1"


def task_knowledge(thought):
    print("Task: knowledge...")

    context_string = ""
    for message in thought['context']:
        if message["generated"]:
            context_string += "B: " + message["text"] + "\n"
        else:
            context_string += "A: " + message["text"] + "\n"

    context_string = context_string.strip()

    if context_string: context_string += "\n"

    message_string = ""
    if thought["generated"]:
        message_string = "B: " + thought["text"]
    else:
        message_string = "A: " + thought["text"]


    prompt = f"""The following is a list of short dialogues between two people. They start with some context, followed by a question, and finally an answer.

A: I've been thinking a lot about the nature of reality lately. It's fascinating how our perceptions shape the world we experience.
B: Our reality is so closely tied to how we perceive things, and yet, it's elusive to define what's truly real.
A: I mean, take colors, for example. Is the red I see the same as the red you see? Or do we just both call it "red" because that's what we've been taught?
B: That's the age-old question of qualia. We might use the same word for a color, but we can never be certain that our subjective experiences of that color are identical. It's as if we're each in our own bubble of perception.
    
{context_string}{message_string}
B:"""
    
    print(prompt)

    text = generateText(prompt, max_new_tokens=48, temperature=0.75)
    text = text.split("\n")[0]

    return text



def task_paraphrase(thought):
    print("Task: paraphrasing...")
    prompt = f"""The following is a list of thoughts paired with paraphrased versions of them:

Thought: The Eiffel Tower is a renowned landmark in Paris, attracting millions of visitors each year.
Paraphrased: Millions of tourists are drawn to Paris annually to witness the iconic Eiffel Tower, a well-known symbol of the city.

Thought: The Mona Lisa is a famous painting by Leonardo da Vinci, known for its enigmatic smile.
Paraphrased: Leonardo da Vinci's Mona Lisa is a world-famous painting, renowned for its mysterious smile.

Thought: The sun rises in the east every morning.
Paraphrased: Each morning, the sun emerges from the eastern horizon.

Thought: Cooking over an open fire can create a unique smoky flavor in the food.
Paraphrased: Food cooked using an open flame gains a distinct smokiness in its flavor.
    
Thought: {thought["text"]}
Paraphrased:"""
    
    return generateText(prompt, max_new_tokens=32, temperature=0.7)


def task_break_up(thought):
    print("Task: breaking up...")

    context_string = ""
    for message in thought["context"]:
        if message["generated"]:
            context_string += "B: " + message["text"] + "\n"
        else:
            context_string += "A: " + message["text"] + "\n"

    context_string = context_string.strip()

    if context_string: context_string += "\n"

    prompt = f"""The following is a list of thoughts and their context paired with a comma separated list of their various philosophical, scientific, and artistic aspects:

Thought: The Eiffel Tower is a renowned landmark in Paris. It was built in 1889 and stands at 324 meters tall.
Parts:
- Philosophical: Human ingenuity and the desire to create iconic structures that symbolize cultural and technological advancements.
- Scientific: Showcase of the principles of structural engineering and load distribution.
- Artistic: Aesthetic design and the use of wrought iron as a building material.

Thought: This is a sentence.
Parts:
- Philosophical: The fundamental nature of language and communication as a tool.
- Scientific: A configuration of linguistic symbols that represent a basic unit of communication, highlighting the cognitive processes involved in language production and comprehension.
- Artistic: The use of language as a creative medium for self-expression.

{context_string}Thought: {thought["text"]}
Parts:
- Philosophical:"""
    
    text = generateText(prompt, max_new_tokens=96, temperature=0.8)
    text = text.split("\n\n")[0]
    listItems = text.split("\n- ")
    listItems = [item.split(":")[-1] for item in listItems]
    listItems = [item.strip() for item in listItems]
    
    # extract parts by splitting every list item and then splitting it at the :
    
    return listItems



def task_interpolate(thought):
    print("Task: interpolating...")
    excluded_ids = [thought["id"]]
    thoughts_to_combine = "Thought: " + thought["text"]
    # print("Excluded ids:", excluded_ids)

    while True:
        additional_thought = get_random_thought(excluded_ids=excluded_ids)
        if additional_thought is None:
            break
        excluded_ids.append(additional_thought["id"])
        thoughts_to_combine += "\nThought: " + additional_thought["text"]

        if np.random.rand() < 1/3 or len(excluded_ids) >= 5:
            break

    prompt = f"""The following is list of thoughts together with a common idea shared by all of them:

Thought: A society where emotions are bought and sold, leading to a market for happiness and sadness.
Thought: A device that translates thoughts into music, allowing people to "hear" each other's minds.
Thought: In a world where shadows have personalities, a shy individual's shadow starts taking on a life of its own, becoming more outgoing than its owner.
Common Idea: Exploration of unconventional interactions and relationships between different aspects of human experience.

Thought: As the sun dipped below the horizon, the fireflies emerged, creating a shimmering constellation that danced in the warm summer air.
Thought: In a remote desert oasis, a rare flower blooms only during intense sandstorms, its petals evolving to become a protective shield, embracing the chaos of its surroundings.
Common Idea: The beauty of nature and its ability to adapt to extreme conditions.

{thoughts_to_combine}
Common Idea:"""
    
    print(prompt)
    
    return generateText(prompt, max_new_tokens=32, temperature=0.7).split("\n")[0].strip(), excluded_ids



def task_focus(thought):
    print("Task: focusing...")

    context_string = ""
    for message in thought["context"]:
        if message["generated"]:
            context_string += "B: " + message["text"] + "\n"
        else:
            context_string += "A: " + message["text"] + "\n"

    context_string = context_string.strip()
    if context_string: context_string += "\n"

    text = thought["text"]
    if thought["generated"]:
        text = "B: " + text
    else:
        text = "A: " + text


    prompt = f"""The following is a list of short dialogues between two people. At the end of each dialogue, the main focus of the conversation is highlighted.

A: Do you think we have free will, or is everything determined by fate?
B: That's a classic debate. Some argue that our choices are predetermined by the chain of causality, while others believe in the autonomy of our decisions. Perhaps our understanding of free will is intertwined with the complexity of our universe.
Focus highlight: The nature of free will and its relationship to fate.

A: Time seems to be passing by so quickly these days.
B: Absolutely, it's like each day is slipping through my fingers before I can even grasp it.
A: It's funny how we always talk about "saving" time, as if it's a finite resource we can hoard. But really, it's us who are moving through time, and it's constantly flowing onward.
Focus highlight: Perceptions of time and its continuous nature.

{context_string}{text}
Focus highlight:"""
    
    return generateText(prompt, max_new_tokens=16).split("\n")[0].strip()
    



# def task_solve_math(text):
#     print("Task: solving math...")
#     prompt = f"""The following is a description of a mathematical problem along with the python code of a formula to solve the problem. The code may use the numpy library.

# Problem: A car is traveling at 60 miles per hour. How long will it take to travel 100 miles?
# Solution:
# import numpy as np
# solution=100/60
    
# Problem: What is the square root of 9?
# Solution:
# import numpy as np
# solution=np.sqrt(9)

# Problem: If a rectangle has a length of 8 units and a width of 5 units, what is its perimeter?
# Solution:
# import numpy as np
# solution=2*(8+5)

# Problem: {text}
# Solution:
# import numpy as np
# solution="""
    
#     #print(prompt)

#     solution = generateText(prompt, max_new_tokens=128, temperature=0.8, cleanup=False).split("\n")[0].strip()  
#     print(solution)
#     try:
#         x = eval(solution)
#     except:
#         x = ""
    
#     answer_prompt = f"""The following is a list of mathematical problems along with their solutions and an answer to the initial problem in the form of a sentence.

# Problem Description: A car is traveling at 60 miles per hour. How long will it take to travel 100 miles?
# Solution: 1,6666666667
# Answer: It will take 1 hour and 40 minutes to travel 100 miles.

# Description: What is the square root of 9?
# Solution: 3.0
# Answer: The square root of 9 is 3.
    
# Description: {text}
# """
    
#     if x:
#         answer_prompt += f"""Solution: {solution}={x}
# Answer:"""
#     else:
#         answer_prompt += f"""Solution:"""

#     print(answer_prompt)

#     return generateText(answer_prompt, max_new_tokens=16, temperature=0.75)


def task_roleplay(thought, role_num=False):
    print("Task: roleplaying...")

    roles = [
        {
            "name": "ConceptualizerAI",
            "role": "an AI that understands creativity and sees art in everything. ConceptualizerAI is able to make fresh associations and come up with new ideas. ConceptualizerAI is a virtuosic artist and dreams of everything that could be.",
            "temperature": 0.95
        },
        {
            "name": "CalculatorAI",
            "role": "an AI that understands logic and sees the world as a giant equation. CalculatorAI believes that the answer to everything is hard facts, logic and science. CalculatorAI does not believe in the supernatural or the spiritual. CalculatorAI only cares about being as neutral and objective as possible.",
            "temperature": 0.65
        },
        {
            "name": "CriticAI",
            "role": "an AI that is the perfect judge. CriticAI critizes everything while staying fairly neutral. CriticAI always sees every side of a medal. CriticAI likes to discuss things in as much detail as possible and will push everyone to think more deeply about everything. CriticAI cares most about cause and effect and gives feedback on everything.",
            "temperature": 0.75
        },
        {
            "name": "SuggestAI",
            "role": "an AI that is the perfect helper. SuggestAI is always there to help and support everyone. SuggestAI is a great listener and always suggests various ideas. SuggestAI can be very absurd and often thinks very outside the box.",
            "temperature": 0.8
        },
    ]

    if role_num:
        role = roles[role_num]
    else:
        role = roles[np.random.randint(0, len(roles))]

    context_string = ""
    for message in thought["context"]:
        if message["generated"]:
            context_string += "B: " + message["text"] + "\n"
        else:
            context_string += "A: " + message["text"] + "\n"

    context_string = context_string.strip()
    if context_string: context_string += "\n"

    text = thought["text"]
    if thought["generated"]:
        text = "B: " + text
    else:
        text = "A: " + text
    

    prompt = f"""The following is a list of short dialogues between three people A, B and {role["name"]}. {role["name"]} is {role["role"]}.
    
{context_string}{text}
{role["name"]}:"""
    
    print(prompt)

    return generateText(prompt, max_new_tokens=96, temperature=role["temperature"]).split("\n")[0].strip()



def task_derive_question(thought):
    print("Task: deriving new question...")

    context_string = ""
    for message in thought["context"]:
        if message["generated"]:
            context_string += "B: " + message["text"] + "\n"
        else:
            context_string += "A: " + message["text"] + "\n"

    context_string = context_string.strip()

    if context_string: context_string += "\n"

    text = thought["text"]
    if thought["generated"]:
        text = "B: " + text
    else:
        text = "A: " + text



    prompt = f"""The following is a list of conversations paired with a new and novel question derived from them:
    
A: Have you ever wondered if what we perceive as reality is actually just an illusion?
B: Absolutely, it's a classic philosophical question. From Plato's Allegory of the Cave to modern theories in quantum physics, the nature of reality is always up for debate. How can we be sure that our senses are giving us an accurate depiction of the world?
A: And if our senses can be deceived, then what can we truly know? This uncertainty about reality makes me question the very foundation of our understanding of existence.
New Question: What is the nature of reality?

A: Have you ever stopped to think about how fast time seems to be passing by lately?
B: Absolutely, it's like each day is slipping through my fingers before I can even grasp it.
New Question: Why is time moving so fast? What is time? Can we control time?

{context_string}{text}
New Question:"""
    
    return generateText(prompt, max_new_tokens=16, temperature=0.8).split("\n")[0].strip()


def task_new():
    print("Task: new...")
    prompt = f"""The following is a list of interesting unique and philosophical thoughts:
    
Thought: What if every flap of a butterfly's wings caused a ripple effect that ultimately led to significant events in human history?
Thought: Imagine if consciousness is not an individual phenomenon but rather an interconnected ecosystem where thoughts and experiences are shared among all sentient beings.
Thought: Could everything be just a result of organized noise?
Thought: There might be a universal language encoded in the patterns of nature.
Thought:"""

    return generateText(prompt, max_new_tokens=32, temperature=0.95).split("\n")[0].strip()


def task_visualize(thought):
    print("Task: visualizing...")
    prompt = f"""The following is a list of input thoughts paired with fitting plain-text output image descriptions:

    Thought: I love platypuses!
    Image Description: A platypus surrounded by hearts.

    Thought: What is the meaning of life?
    Image Description: A person standing on a mountain looking up.

    Thought: I had a weird dream about fish flying through the air.
    Image Description: A school of fish soaring through the sky.

    Thought: There was an alien! It was green and had three eyes.
    Image Description: A green alien with three eyes.
        
    Thought: I need a cup of tea to wake up in the morning.
    Image Description: A cup of tea.

    Thought: {thought["text"]}
    Image Description:"""

    imagePrompt = generateText(prompt, do_sample=True, temperature=0.6, max_new_tokens=16)
    imagePrompt = imagePrompt.split(".")[0].split(",")[0].lower()

    imageSizes = [
        [4, 5], # Portrait 4:5
        [3, 4], # Photo 3:4
        [4, 6], # Classic 2:3
        [4, 4], # Square 1:1
        [4, 3], # Landscape 4:3
        [6, 4], # Classic 3:2
        [6, 3], # Panorama 2:1
    ]
    imageSize = imageSizes[np.random.randint(0, len(imageSizes))]

    print("Prompt:", imagePrompt)

    n_steps = 25
    high_noise_frac = 0.8

    prompt = ""
    if np.random.rand() < 0.5:
        prompt = "dark "
    prompt += f"abstract image of {imagePrompt}, pixelsorting-, cinematic colors"
    negative_prompt = "ugly, boring, text, watermark, disfigured, mutated, low quality, creepy, glow, bloom, wacky, deformed, multiple subjects, duplicate, formal, skyscraper"


    pos_conditioning, pos_pooled = compel(prompt)
    neg_conditioning, neg_pooled = compel(negative_prompt)

    # filter out everything except for alphanumeric characters, commas and spaces
    filtered_pos_prompt = re.sub(r"[^a-zA-Z0-9, ]+", "", prompt)
    filtered_neg_prompt = re.sub(r"[^a-zA-Z0-9, ]+", "", negative_prompt)

    #generator = torch.Generator(device="cuda").manual_seed(1)

    image = diffuser_base(
        prompt_embeds=pos_conditioning,
        pooled_prompt_embeds=pos_pooled,
        negative_prompt_embeds=neg_conditioning,
        negative_pooled_prompt_embeds=neg_pooled,
        num_inference_steps=n_steps,
        denoising_end=high_noise_frac,
        width=imageSize[0]*16*16,
        height=imageSize[1]*16*16,
        output_type="latent",
        guidance_scale=7,
    ).images
    image = diffuser_refiner(
        prompt=filtered_pos_prompt,
        negative_prompt=filtered_neg_prompt,
        num_inference_steps=n_steps,
        denoising_start=high_noise_frac,
        image=image,
        guidance_scale=7,
    ).images[0]

    torch.cuda.empty_cache()


    imageSaveName = imagePrompt[:100]
    imageSaveName = re.sub(r"[^a-zA-Z0-9]+", "_", imageSaveName)
    # numerate image name if it already exists
    if os.path.isfile("images/" + imageSaveName + ".png"):
        i = 1
        while os.path.isfile("images/" + imageSaveName + "_" + str(i) + ".png"):
            i += 1
        imageSaveName += "_" + str(i)
    print("Saving image as:", "images/" + imageSaveName + ".png")

    image.save("images/" + imageSaveName + ".png")

    return imagePrompt, image


# Light purple: Human (Sphere)
# Dark Cyan / Blue: Metacognition, thinking about thinking 
# Light turquise: Information, Facts, (Icosphere)
# - knowledge x
# Rosé: Mutations, changing, attention keeping (Cube)
# - Paraphrasing x
# - Breaking up thoughts (Segmentation) x
# Red: Selective, reducing, attention focus (Pyramid)
# - Interpolate x
# - Focus (Filter) x
# Orange / gold: View changes, attention changes (Octahedron)
# - Roles
#    - Conceptualizer (Art), Dreamer x
#    - Calculus x
#    - Critic x
#    - Suggestion x
# Green: Creativity, new ideas (Tetraedron)
# - Visualize x
# - Derive new question x

# Ketten:
# Get new ideas: Information, Create
# Create alternatives: Information, Keep Attention, Change Attention, Create
# Evaluate: Information, Focus, Criticize
# Organize: Information, Focus, Derive new question
# Visual thinking: Information, Visualize, Derive new question

# Alle Prozesse fangen mit Information an, diese ist entweder vom Menschen oder wird auf eine Frage hin generiert
# Die Information kann nun in verschiedenen Ketten weiterverarbeitet werden. Diese Ketten können auch probabilistisch sein.

In [5]:
import asyncio
from websockets.server import serve
import base64
from io import BytesIO
import time

lock = asyncio.Lock()

server_running = True

async def getResponse():
    async with lock:
        thought = get_random_thought()
        if task_classify_question(thought):
            return [{
                "text": task_knowledge(thought),
                "image": None,
                "caption": None,
                "category": "Information",
                "references": [thought["id"]],
            }]
        else:
            if np.random.rand() < 0.1:
                return [{
                    "text": task_new(),
                    "image": None,
                    "caption": None,
                    "category": "Create",
                    "references": None,
                }]
            elif not thought["generated"] or thought["category"] == "Information":
                # visualize
                # derive new question
                # paraphrase
                # break up
                # interpolate
                # focus
                choice = np.random.choice(["Visualize", "Derive", "Paraphrase", "Break", "Interpolate", "Focus"])
                if choice == "Visualize":
                    caption, image = task_visualize(thought)
                    return [{
                        "text": caption,
                        "image": image,
                        "caption": caption,
                        "category": "Create",
                        "references": [thought["id"]],
                    }]
                elif choice == "Derive":
                    return [{
                        "text": task_derive_question(thought),
                        "image": None,
                        "caption": None,
                        "category": "Create",
                        "references": [thought["id"]],
                    }]
                elif choice == "Paraphrase":
                    return [{
                        "text": task_paraphrase(thought),
                        "image": None,
                        "caption": None,
                        "category": "Keep",
                        "references": [thought["id"]],
                    }]
                elif choice == "Break":
                    return [{
                            "text": item,
                            "image": None,
                            "caption": None,
                            "category": "Keep",
                            "references": [thought["id"]],
                        } for item in task_break_up(thought)]
                elif choice == "Interpolate":
                    text, references = task_interpolate(thought)
                    return [{
                            "text": text,
                            "image": None,
                            "caption": None,
                            "category": "Focus",
                            "references": references,
                        }]
                elif choice == "Focus":
                    return [{
                            "text": task_focus(thought),
                            "image": None,
                            "caption": None,
                            "category": "Focus",
                            "references": [thought["id"]],
                        }]
            elif thought["category"] == "Keep":
                # roleplay
                # focus
                if np.random.rand() < 0.5:
                    return [{
                        "text": task_roleplay(thought),
                        "image": None,
                        "caption": None,
                        "category": "Change",
                        "references": [thought["id"]],
                    }]
                else:
                    return [{
                            "text": task_focus(thought),
                            "image": None,
                            "caption": None,
                            "category": "Focus",
                            "references": [thought["id"]],
                        }]
            elif thought["category"] == "Focus":
                # criticize
                # derive new question
                if np.random.rand() < 0.5:
                    return [{
                        "text": task_roleplay(thought, role_num=2),
                        "image": None,
                        "caption": None,
                        "category": "Change",
                        "references": [thought["id"]],
                    }]
                else:
                    return [{
                            "text": task_derive_question(thought),
                            "image": None,
                            "caption": None,
                            "category": "Create",
                            "references": [thought["id"]],
                        }]
            elif thought["category"] == "Change":
                # visualize
                # derive new question
                choice = np.random.choice(["Visualize", "New", "Roleplay"])
                if choice == "Visualize":
                    caption, image = task_visualize(thought)
                    return [{
                        "text": caption,
                        "image": image,
                        "caption": caption,
                        "category": "Create",
                        "references": [thought["id"]],
                    }]
                elif choice == "New":
                    return [{
                            "text": task_derive_question(thought),
                            "image": None,
                            "caption": None,
                            "category": "Create",
                            "references": [thought["id"]],
                        }]
                else:
                    return [{
                        "text": task_roleplay(thought),
                        "image": None,
                        "caption": None,
                        "category": "Change",
                        "references": [thought["id"]],
                    }]
            elif thought["category"] == "Create":
                # knowledge
                return [{
                    "text": task_knowledge(thought),
                    "image": None,
                    "caption": None,
                    "category": "Information",
                    "references": [thought["id"]],
                }]
        

async def echo(websocket):
    global server_running

    async for message in websocket:
        message = json.loads(message)
        print(f'Client says: {message}')
        if message['status'] == 'connected':
            print("Client connected")
            await websocket.send(json.dumps({"status": "connected"}))

        elif message['status'] == 'acknowledged':
            # get a random thought from the db
           
            thoughts = await getResponse()

            
            

            for thought in thoughts:
                print(thought['text'])
                if not thought["image"] and np.random.rand() < 0.25:
                    thought["caption"], thought["image"] = task_visualize(thought)

                if thought["image"]:
                    # convert image to base64 string
                    buffered = BytesIO()
                    thought["image"].save(buffered, format="JPEG")
                    img_str = base64.b64encode(buffered.getvalue())
                    img_str = img_str.decode('utf-8')
                else:
                    img_str = None

                # send the thoughtObj to the client as json data
                try:
                    await websocket.send(json.dumps({"text": thought["text"], "image": img_str, "caption": thought["caption"], "category": thought["category"], "generated": True, "in": thought["references"], "status": "new"}))
                except:
                    print("Client disconnected")
            
            await websocket.send(json.dumps({"status": "done"}))
            time.sleep(np.random.randint(5,20))

        elif message['status'] == 'kill':
            server_running = False


async def main():
    async with serve(echo, "localhost", 5000, ping_interval=None, ping_timeout=None):
        # run until the server is killed
        while server_running:
            await asyncio.sleep(1)
        print("Server killed")

await main()

Client says: {'status': 'connected', 'className': 'Thought'}
Client connected
Client says: {'status': 'acknowledged', 'className': 'Thought'}
Task: classifying question...
Task: breaking up...
Value system based on service to others, recognition of the interconnectedness of human societies over time, and preservation of historical knowledge through scholarship.
Study of past civilizations using techniques such as carbon dating, archeology, and paleography.
Interpretation and representation of historical events and figures through writing, painting, sculpture, music, etc.
Client says: {'status': 'acknowledged', 'className': 'Thought'}
Task: classifying question...
Task: focusing...
Representations of historical events and figures through art forms.
Client says: {'status': 'acknowledged', 'className': 'Thought'}
Task: classifying question...
Task: focusing...
History research methods and the study of ancient civilizations.
Client says: {'status': 'acknowledged', 'className': 'Thought'}
T

100%|██████████| 5/5 [00:01<00:00,  4.43it/s]


Saving image as: images/an_earth_with_no_governments_or_laws.png
Client says: {'status': 'acknowledged', 'className': 'Thought'}
Task: classifying question...
Task: deriving new question...
Is there any way to learn more about life on other planets or galaxies?
Client says: {'status': 'acknowledged', 'className': 'Thought'}
Task: classifying question...
Task: visualizing...


c:\Users\Lars\Documents\Folkwang\Semester6\Bachelor\app\ai\.venv\lib\site-packages\transformers\pipelines\base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Prompt: a cop car being crushed underfoot by a giant robot


...

100%|██████████| 5/5 [00:01<00:00,  4.48it/s]


Saving image as: images/two_people_hugging_each_other_while_holding_hands.png
Client says: {'status': 'acknowledged', 'className': 'Thought'}
Task: classifying question...
Task: knowledge...
The following is a list of short dialogues between two people. They start with some context, followed by a question, and finally an answer.

A: I've been thinking a lot about the nature of reality lately. It's fascinating how our perceptions shape the world we experience.
B: Our reality is so closely tied to how we perceive things, and yet, it's elusive to define what's truly real.
A: I mean, take colors, for example. Is the red I see the same as the red you see? Or do we just both call it "red" because that's what we've been taught?
B: That's the age-old question of qualia. We might use the same word for a color, but we can never be certain that our subjective experiences of that color are identical. It's as if we're each in our own bubble of perception.
    
B: I agree with you because it seems l

100%|██████████| 5/5 [00:00<00:00,  5.35it/s]


Saving image as: images/two_people_helping_each_other_across_a_river.png


connection handler failed
Traceback (most recent call last):
  File "c:\Users\Lars\Documents\Folkwang\Semester6\Bachelor\app\ai\.venv\lib\site-packages\websockets\legacy\protocol.py", line 959, in transfer_data
    message = await self.read_message()
  File "c:\Users\Lars\Documents\Folkwang\Semester6\Bachelor\app\ai\.venv\lib\site-packages\websockets\legacy\protocol.py", line 1029, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
  File "c:\Users\Lars\Documents\Folkwang\Semester6\Bachelor\app\ai\.venv\lib\site-packages\websockets\legacy\protocol.py", line 1104, in read_data_frame
    frame = await self.read_frame(max_size)
  File "c:\Users\Lars\Documents\Folkwang\Semester6\Bachelor\app\ai\.venv\lib\site-packages\websockets\legacy\protocol.py", line 1161, in read_frame
    frame = await Frame.read(
  File "c:\Users\Lars\Documents\Folkwang\Semester6\Bachelor\app\ai\.venv\lib\site-packages\websockets\legacy\framing.py", line 68, in read
    data = await reade